In [1]:
import tensorflow as tf
from android_env.proto.a11y import android_accessibility_forest_pb2
import json

2025-04-20 19:21:20.881919: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-20 19:21:20.997728: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-20 19:21:20.997799: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-20 19:21:20.997868: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-20 19:21:21.019027: I tensorflow/core/platform/cpu_feature_g

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

2025-04-20 19:21:23.327814: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/home/cs/anaconda3/envs/appvlm_tf/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.30.2 at android_env/proto/a11y/android_accessibility_forest.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/home/cs/anaconda3/envs/appvlm_tf/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.30.2 at android_env/proto/a11y/android_accessibility_window_info.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/home/cs/anaconda3/envs/appvlm_tf/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.30.2 at android_env/pr

In [6]:
dataset = tf.data.TFRecordDataset("./dataset/android_control/android_control-00011-of-00020",compression_type="GZIP")

In [7]:
for raw_record in dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    # 提取字段
    features = example.features.feature
    accessibility_bytes = example.features.feature["accessibility_trees"].bytes_list.value[0]
    forest = android_accessibility_forest_pb2.AndroidAccessibilityForest().FromString(accessibility_bytes)
    

In [40]:
from android_env.proto.a11y import android_accessibility_forest_pb2

# 查找包含点击点的 window
def find_target_window(windows, x, y):
    for window in windows:
        bounds = window.bounds_in_screen
        if bounds.left <= x < bounds.right and bounds.top <= y < bounds.bottom:
            return window
    return None

# 获取该 window 中的所有节点
def collect_nodes_from_window(window):
    return list(window.tree.nodes)

# 判断一个点是否在 bounds 中
def is_point_inside_bounds(x, y, bounds):
    return bounds.left <= x < bounds.right and bounds.top <= y < bounds.bottom

# 找出所有包含点击点的节点
def find_nodes_containing_point(nodes, x, y):
    return [node for node in nodes if is_point_inside_bounds(x, y, node.bounds_in_screen)]

# 判断是否具有语义信息
def has_semantic_info(node):
    return bool(
        getattr(node, "view_id_resource_name", "") or
        getattr(node, "content_description", "") or
        getattr(node, "text", "")
    )

def find_descendant_with_text(node, all_nodes):
    """
    找出该节点的所有后代中最“有语义”的那个（含 text / content_description / resource_id），
    优先 text > content_description > view_id_resource_name
    """
    best_node = None
    best_score = -1

    def dfs(n):
        nonlocal best_node, best_score
        score = 0
        if getattr(n, "text", ""):
            score += 3
        if getattr(n, "content_description", ""):
            score += 2
        if getattr(n, "view_id_resource_name", ""):
            score += 1
        if score > best_score:
            best_score = score
            best_node = n
        for cid in getattr(n, "child_ids", []):
            child = next((x for x in all_nodes if x.unique_id == cid), None)
            if child:
                dfs(child)

    dfs(node)
    return best_node if best_score > 0 else None


# 提取结构化信息
def extract_node_info(node):
    return {
        "unique_id": node.unique_id,
        "class": node.class_name,
        "package": node.package_name,
        "text": getattr(node, "text", ""),
        "content description": getattr(node, "content_description", ""),
        "resource name": getattr(node, "view_id_resource_name", ""),
        "is_clickable": getattr(node, "is_clickable", False),
    }

# 从点击坐标构建 click action
def convert_click_to_element_action(forest_bytes, x, y):
    forest = android_accessibility_forest_pb2.AndroidAccessibilityForest()
    forest.ParseFromString(forest_bytes)

    target_window = find_target_window(forest.windows, x, y)
    if target_window is None:
        print(f"❌ 没有找到包含点击点 ({x}, {y}) 的窗口")
        return None

    all_nodes = collect_nodes_from_window(target_window)
    matched_nodes = find_nodes_containing_point(all_nodes, x, y)

    print(f"\n🟡 找到 {len(matched_nodes)} 个匹配点 ({x}, {y}) 的节点")
    for node in matched_nodes:
        b = node.bounds_in_screen
        print(f"  - ID={node.unique_id} Bounds=({b.left}, {b.top}, {b.right}, {b.bottom}) Class={node.class_name}")

    if not matched_nodes:
        return None

    # 优先选择最深的节点作为目标（比如 unique_id=43）
    target_node = max(matched_nodes, key=lambda n: getattr(n, "depth", 0))

    # 尝试向下找更具语义的子节点（比如 unique_id=71）
    refined_node = find_descendant_with_text(target_node, all_nodes)
    final_node = refined_node if refined_node else target_node

    print(f"\n🎯 最终选中的节点 ID={final_node.unique_id}, Class={final_node.class_name}")
    return {
        "action type": "click",
        "target element": extract_node_info(final_node)
    }

In [44]:
features = example.features.feature
x, y = 447, 2216 # 假设点击坐标

result = convert_click_to_element_action(
    forest_bytes=features['accessibility_trees'].bytes_list.value[4],
    x=x,
    y=y
)

print(result)


🟡 找到 20 个匹配点 (447, 2216) 的节点
  - ID=0 Bounds=(0, 0, 1080, 2400) Class=android.widget.FrameLayout
  - ID=1 Bounds=(0, 0, 1080, 2400) Class=android.widget.FrameLayout
  - ID=4 Bounds=(0, 0, 1080, 2400) Class=android.widget.FrameLayout
  - ID=5 Bounds=(0, 0, 1080, 2400) Class=android.widget.FrameLayout
  - ID=6 Bounds=(0, 0, 1080, 2400) Class=android.widget.FrameLayout
  - ID=7 Bounds=(0, 0, 1080, 2400) Class=android.widget.RelativeLayout
  - ID=10 Bounds=(0, 2095, 1080, 2337) Class=android.widget.FrameLayout
  - ID=11 Bounds=(0, 2185, 1080, 2337) Class=android.widget.ImageView
  - ID=15 Bounds=(0, 2095, 1080, 2337) Class=android.view.ViewGroup
  - ID=22 Bounds=(0, 2095, 1080, 2337) Class=android.widget.FrameLayout
  - ID=23 Bounds=(0, 2116, 1080, 2316) Class=android.widget.RelativeLayout
  - ID=29 Bounds=(0, 2095, 1080, 2337) Class=android.widget.ImageView
  - ID=30 Bounds=(0, 2116, 1080, 2316) Class=android.widget.LinearLayout
  - ID=37 Bounds=(0, 2116, 1080, 2316) Class=android.widget